In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm


In [ ]:
class HoverNet(nn.Module):
    def __init__(self):
        super(HoverNet, self).__init__()
        # Define your HoverNet architecture here
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.conv2(x)
        return x

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform

        self.image_filenames = sorted(os.listdir(image_folder))
        self.mask_filenames = sorted(os.listdir(mask_folder))

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_filenames[idx])
        mask_name = os.path.join(self.mask_folder, self.mask_filenames[idx])

        image = Image.open(img_name).convert("RGB")
        mask = Image.open(mask_name).convert("L")  # Assuming masks are grayscale

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

In [ ]:
# Define transforms if needed
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images and masks to the same size
    transforms.ToTensor(),  # Convert images and masks to tensors
])


In [ ]:
# Define dataset and dataloader
image_folder = '/content/drive/MyDrive/Hover/Images'
mask_folder = '/content/drive/MyDrive/Hover/Mask'
batch_size = 8
learning_rate = 0.0001  # Set your desired learning rate value

dataset = CustomDataset(image_folder=image_folder, mask_folder=mask_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Instantiate your HoverNet model
hovernet = HoverNet()

# Define the loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(hovernet.parameters(), lr=learning_rate)


In [ ]:
num_epochs = 200  # Define the number of epochs for training
from PIL import Image


In [ ]:
for epoch in range(num_epochs):
    hovernet.train()  # Set model to training mode
    for images, masks in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
        optimizer.zero_grad()  # Zero the gradients
        outputs = hovernet(images)  # Forward pass
        loss = criterion(outputs, masks.float())  # Convert masks to float
         # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights


Epoch 200/200: 100%|██████████| 3/3 [00:01<00:00,  2.19it/s]
